In [5]:
import numpy as np
import pandas as pd
import math

In [6]:
df = pd.read_csv("./final_cleaned_2.csv")

In [7]:
df.head()

,id,adult,belongs_to_collection,genres,overview,popularity,production_companies,release_date,runtime,spoken_languages,title,vote_average,vote_count,keywords,cast,crew
0,862,0,10194.0,"[16, 35, 10751]","Led by Woody, Andy's toys live happily in his ...",21.946943,[3],1995,81.0,[1],Toy Story,7.7,5415.0,"[931, 4290, 5202, 6054, 9713, 9823, 165503, 17...","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[7879, 12891, 7, 12892, 12893, 12894, 12895, 1..."
1,8844,0,NaN,"[12, 14, 10751]",When siblings Judy and Peter discover an encha...,17.015539,"[559, 2550, 10201]",1995,104.0,"[1, 2]",Jumanji,6.9,2413.0,"[10090, 10941, 15101, 33467, 158086, 158091]","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[511, 876, 1729, 4945, 4951, 4952, 8023, 9967,..."
2,15602,0,119050.0,"[10749, 35]",A family wedding reignites the ancient feud be...,11.712900,"[6194, 19464]",1995,101.0,[1],Grumpier Old Men,6.5,92.0,"[1495, 12392, 179431, 208510]","[6837, 3151, 13567, 16757, 589, 16523, 7166]","[26502, 16837, 16837, 1551320]"
3,31357,0,NaN,"[35, 18, 10749]","Cheated on, mistreated and stepped on, the wom...",3.859495,[306],1995,127.0,[1],Waiting to Exhale,6.1,34.0,"[818, 10131, 14768, 15160, 33455]","[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[2178, 5144, 5144, 21968, 70592, 111118, 11111..."
4,11862,0,96871.0,[35],Just when George Banks has recovered from his ...,8.387519,"[5842, 9195]",1995,106.0,[1],Father of the Bride Part II,5.7,173.0,"[1009, 1599, 2246, 4995, 5600, 10707, 13149, 3...","[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[37, 5506, 17698, 17698, 26160, 56106, 68755]"


In [8]:
df.iloc[0]

id                                                                     862
adult                                                                    0
belongs_to_collection                                              10194.0
genres                                                     [16, 35, 10751]
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
production_companies                                                   [3]
release_date                                                          1995
runtime                                                               81.0
spoken_languages                                                       [1]
title                                                            Toy Story
vote_average                                                           7.7
vote_count                                                          5415.0
keywords                 

In [9]:
df.columns

Index(['id', 'adult', 'belongs_to_collection', 'genres', 'overview',
       'popularity', 'production_companies', 'release_date', 'runtime',
       'spoken_languages', 'title', 'vote_average', 'vote_count', 'keywords',
       'cast', 'crew'],
      dtype='object')

In [20]:
df = df[['id','adult','belongs_to_collection','genres','production_companies','spoken_languages','keywords','cast','crew']]

In [21]:
def score_similarity(a_,b_,attr):
    if attr == 'adult':
        return int(a_==b_) 
    elif attr == 'belongs_to_collection':
        if a_ == b_:
            return 1
        else:
            return 0
    elif attr in ['genres','production_companies','spoken_languages','keywords','crew','cast']:
        a_ = set(eval(a_))
        b_ = set(eval(b_))
        if len(a_|b_) == 0:
            return 0
        score = len(a_&b_)/len(a_|b_)
        return score
    elif attr in ['popularity','runtime','release_date','vote_average']:
        score = 1/abs(a_-b_)
        return score
    else:
        return 0

In [49]:
def get_similarity(a,b):
    similarity = []
    if b in df.id.values and a in df.id.values:
        a_ = df.iloc[df.index[df.id == a][0]]
        b_ = df.iloc[df.index[df.id == b][0]]
        
        for i in list(a_.index)[1:]:
            similarity.append(score_similarity(a_[i],b_[i],i))

        return np.array(similarity).mean()
    return 0

In [34]:
# don't consider vote count, title and overview
get_similarity(67758,8844)

0.1875

In [24]:
df.columns

Index(['id', 'adult', 'belongs_to_collection', 'genres',
       'production_companies', 'spoken_languages', 'keywords', 'cast', 'crew'],
      dtype='object')

In [61]:
df_recommendations = pd.read_csv("./ItemKNN_nn=50_sim=cosine_imp=standard_bin=False_shrink=0_norm=True_asymalpha=_tvalpha=_tvbeta=_rweights=.tsv",sep='\t',names=['user_id','movie_id','score'])
df_1 = df_recommendations.iloc[:10]
df_1['score'] = df_1['score']/df_1['score'].max()

/var/folders/1f/gmncyz4s6hq38dzhq0jbccn40000gn/T/ipykernel_28651/3615608960.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['score'] = df_1['score']/df_1['score'].max()


In [62]:
def compute_dissimilarity(selected_ids, movie_id):
    """Intra-list diversity using MMR"""
    dissimilarity = np.mean([1 - get_similarity(s_id, movie_id) for s_id in selected_ids])
    return dissimilarity


def diversity_using_mmr(recommendations, top_n, lambda_param=0.7):
    # recommendations['song_id'] = recommendations['song_id'].astype(str)
    recommendations_list = recommendations.to_dict('records')

    selected = []
    remaining = recommendations_list.copy()

    while len(selected) < top_n and remaining:
        mmr_scores = []

        for movie in remaining:
            movie_id = movie['movie_id']

            if not selected:
                mmr_score = lambda_param * movie['score']
            else:
                # Compute dissimilarity to already selected songs
                dissimilarity = compute_dissimilarity(selected, movie_id)
                mmr_score = lambda_param * movie['score'] - (1 - lambda_param) * dissimilarity
                # print(str(dissimilarity) + " " + str(mmr_score))
           
            mmr_scores.append(mmr_score)

        # Select song with highest MMR score
        max_mmr_idx = np.argmax(mmr_scores)
        selected_movie = remaining.pop(max_mmr_idx)
        selected.append(selected_movie['movie_id'])

    return pd.DataFrame(selected)

In [63]:
diversity_using_mmr(df_1,10)

,0
0,1266
1,1965
2,2243
3,2728
4,2791
5,2108
6,1228
7,1292
8,3361
9,1299


In [64]:
df_1

,user_id,movie_id,score
0,1,1266,1.000000
1,1,1965,0.928717
2,1,2243,0.903481
3,1,2728,0.804829
4,1,1228,0.799046
5,1,2791,0.796977
6,1,1292,0.777947
7,1,2108,0.764875
8,1,3361,0.761733
9,1,1299,0.754398
